In [20]:
import requests 
from bs4 import BeautifulSoup

link = 'https://sweet.tv/movies/usi-filmi'


headers = {
    'authority': 'sweet.tv',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '"Google Chrome";v="87", " Not;A Brand";v="99", "Chromium";v="87"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'uk',
    'cookie': '_gcl_au=1.1.742482539.1635872137; _gid=GA1.2.578436693.1635872138; _fbp=fb.1.1635872137547.730584858; SWEET.TV_session=eyJpdiI6IndPYUJXem1iQS9oVHJndTVtbW9sRXc9PSIsInZhbHVlIjoiMlNkblhzRzg2MWRxNXM4TlEyaEJkK2tOQmY4WCtlTmxtYmpEbWI2V09BOXdQMUtDK0tpTW5HM2crc2o1YnBUNnh4S3pFWUp6WWp6RHM3YW5aQVhPZEZjQnErMmVWa1lsK3ZXZ0lQdjNGamFDZzk2bjBpUjQ2VTVqbjlFaEM5UUQiLCJtYWMiOiJhMDEwZWJkZTBlMWFhNDk4N2JkMmU2YzAyZTAxNDIwY2JmYWY2Nzk4ZmRlMGVmMjk1MGRjMmY1MjU2NDM3N2U3IiwidGFnIjoiIn0%3D; _ga_809044V2BB=GS1.1.1635872137.1.1.1635873541.60; _ga=GA1.2.1169525567.1635872138',
}

response = requests.get('https://sweet.tv/movies/usi-filmi', headers=headers)

def get_soup(link):
    soup = None
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.text, 'lxml')
    except Exception as e:
        print(e)
    
    return soup
        
def get_films(page):
    soup = get_soup(link+f'/page/{page}')
    if soup != None:
        for film in soup.find_all('li', class_="film-item-container"):
            name = film.find('h4').text
            href = film.a.get('href')
            des = film.find('p').text
            
            insert_film(name, href, des)
    

def qty_of_pages(link):
    soup = get_soup(link)
    if soup != None:
        return soup.find('ul', class_='pagination pages_of_search').find_all()[-1].text
    
if __name__ == '__main__':
    qty  = int(qty_of_pages(link))
    for i in range(1,3):
        get_films(i)

In [14]:
import sqlite3

sql_text = """CREATE TABLE films ( 
                                name TEXT NOT NULL,
                                href TEXT NOT NULL,
                                des  TEXT NOT NULL) """

conn = sqlite3.connect('films.sqlite3')
curs = conn.cursor()
curs.execute(sql_text)
conn.close()

In [19]:
def insert_film(name, href, des):
    sql_text = """INSERT INTO films (name, href, des) VALUES (?, ?, ?)"""

    conn = sqlite3.connect('films.sqlite3')
    curs = conn.cursor()
    curs.execute(sql_text, (name, href, des))
    conn.commit()
    conn.close()

# insert_film()

In [27]:
def get_all_film():
    sql_text = """SELECT * FROM films """

    conn = sqlite3.connect('films.sqlite3')
    curs = conn.cursor()
    curs.execute(sql_text)
    for film in curs.fetchall():
        print(film)
    conn.close()
    
get_all_film()

('A', 'B', 'C')
('Агент Джонні Інгліш: Перезапуск', 'https://sweet.tv/movie/16715-agent-dzhonni-inglish-perezapusk', 'Комедії, Бойовики, Пригоди')
('Мішель Вальян: Жага швидкості', 'https://sweet.tv/movie/17275-mishel-valyan-zhaga-shvidkosti', 'Трилери')
('10 клас', 'https://sweet.tv/movie/19648-10-klas', 'Онлайн школа')
('Без обличчя', 'https://sweet.tv/movie/20941-bez-oblichchya', 'Бойовики, Фантастика, Трилери, Кримінал')
('Бартон Фінк', 'https://sweet.tv/movie/21890-barton-fink', 'Комедії, Драми, Трилери')
('Усі собаки потрапляють до раю 2', 'https://sweet.tv/movie/10926-usi-sobaki-potraplyayut-do-rayu-2', 'Пригоди, Сімейні')
('Змивайся', 'https://sweet.tv/movie/10759-zmivaysya', 'Сімейні')
('Простачка', 'https://sweet.tv/movie/16841-prostachka', 'Комедії, Мелодрами, Ultra HD')
('Параноя', 'https://sweet.tv/movie/2172-paranoya', 'Драми, Трилери')
('Повільно на захід', 'https://sweet.tv/movie/3713-povilno-na-zahid', 'Драми, Трилери')
('Пори року в дикій природі Америки', 'https://sw